# 챗봇 만들기

In [1]:
import math
import numpy as np
import pandas as pd
import re
import os

# telegram
import telegram as tel
from telegram.ext import Updater
from telegram.ext import MessageHandler, Filters
from telegram import chat

# 크롤링
from bs4 import BeautifulSoup

import requests
from selenium import webdriver

# gpt함수
import torch
import urllib.request
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel

# 필요 데이터

In [3]:
kospi_list = pd.read_csv('./data/recent_kospi_list.csv')
corp_list = kospi_list['Name']
content_lst = ['최신뉴스','최근뉴스']
reco_lst1 = ['살만한 주식 뭐 있어?','살만한 종목 뭐 있어?']

# 함수

## 크롤링 함수: crawl_news()

In [15]:
def crawl_news(corp,page=1,num=5,bgn_date='2022.03.01',end_date='2022.03.30'):
    
    bgn_date1 = bgn_date
    bgn_date2 = bgn_date.replace('.','')
    end_date1 = end_date
    end_date2 = end_date.replace('.','')
    
    title_lst = []
    url_lst = []
    date_lst = []

    for pg in range(1,page+1):

        page_num = pg *10 - 9

        url = f'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={corp}&sort=0&photo=0&field=0&pd=3&ds={bgn_date1}&de={end_date1}&cluster_rank=24&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{bgn_date2}to{end_date2},a:all&start={page_num}'
        res = requests.get(url)
        soup = BeautifulSoup(res.text , 'html.parser')
        lis = soup.select('#main_pack > section > div > div.group_news > ul>li')

        for li in lis:
            #제목
            title = li.select('div.news_wrap.api_ani_send > div > a')[0].text

            title_lst.append(title)

            # url
            url_path = li.select('div.news_wrap.api_ani_send > div > a')[0]['href']
            url_lst.append(url_path)

            #날짜

            if len(li.select('div.news_info > div.info_group > span'))==1:
                date = li.select('div.news_info > div.info_group > span')[0].text
                date_lst.append(date)


            if len(li.select('div.news_info > div.info_group > span'))==2:
                date = li.select('div.news_info > div.info_group > span')[1].text
                date_lst.append(date)
    
    df = pd.DataFrame({'날짜':date_lst,'뉴스제목':title_lst,'url':url_lst})
    
    output_result = ''
    for i in range(len(df)):
        title = df['뉴스제목'].iloc[i]
        news_url = df['url'].iloc[i]
        output_result += title + "\n" + news_url + "\n\n"
        if i == num:
            break
        
    return output_result

In [16]:
crawl_news('삼성전자')

'SSD 1위 삼성전자, 2위 WD와 MOU 체결…"기술 평준화 위한 협력"\nhttp://www.newsis.com/view/?id=NISX20220330_0001813003&cID=13001&pID=13000\n\n알아서 척척 \'캄테크\' 강조한 한종희…삼성 TV 신제품 공개\nhttps://www.hankyung.com/economy/article/202203301924g\n\n현대重·삼성전자 헬스케어 \'맞손\'..입는 의료기기 협력한다\nhttp://news.mt.co.kr/mtview.php?no=2022033010060122966\n\n신보, 삼성전자 냉장가전 협력사에 181억원 보증지원\nhttp://yna.kr/AKR20220330048700002?did=1195m\n\n삼성전자, 태국에 초대형 마이크로 LED ‘더 월’ 설치\nhttps://biz.chosun.com/it-science/ict/2022/03/29/ZL3KVAX2OZBMBPWBGKX5GL4XYE/?utm_source=naver&utm_medium=original&utm_campaign=biz\n\n[단독]검찰, 삼성웰스토리·삼성전자 본사 3일째 연달아 압수수색\nhttps://www.seoul.co.kr/news/newsView.php?id=20220330500036&wlog_tag3=naver\n\n'

## gpt함수: finance_gpt(user_text)

In [11]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

model1 = torch.load('./data/GPT2_model/gpt_finance.pth')
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [12]:
def finance_gpt(user_text):
    
    with torch.no_grad():
        answer = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + user_text + SENT + A_TKN + answer)).unsqueeze(dim=0)
            pred = model1(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            answer += gen.replace("▁", " ").strip()
    return answer

In [13]:
finance_gpt('주식 해도 될까?')

'능력이있으면하면되죠'

In [3]:
model1 = torch.load('./data/GPT2_model/gpt_finance.pth')
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 

with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            pred = model1(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


user > 주식 투자할까?
Chatbot > 어떻게 하느냐에 따라 다를 것 같아요
user > 시가총액이뭐야
Chatbot > 금융시장에서 유통되는 주식이 현재거래가격기준으로 총 얼마의 가치를 가지는지 나타내는 지표입니다
user > quit


# 코드

In [1]:
import math
import numpy as np
import pandas as pd
import re
import os

# telegram
import telegram
from telegram.ext import Updater
from telegram.ext import MessageHandler, Filters
from telegram import chat

# 크롤링
from bs4 import BeautifulSoup
import requests
from selenium import webdriver

# gpt함수
import torch
import urllib.request
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel

Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

model1 = torch.load('./data/GPT2_model/gpt_finance.pth')
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 

kospi_list = pd.read_csv('./data/recent_kospi_list.csv')
corp_list = kospi_list['Name']
content_lst = ['최신뉴스','최근뉴스']

reco_lst1 = ['살만한 주식 뭐 있어?','살만한 종목 뭐 있어?']

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [2]:
 ######## 크롤링 관련 함수 ########

def crawl_news(corp,page=1,num=5,bgn_date='2022.03.01',end_date='2022.03.30'):
    
    bgn_date1 = bgn_date
    bgn_date2 = bgn_date.replace('.','')
    end_date1 = end_date
    end_date2 = end_date.replace('.','')
    
    title_lst = []
    url_lst = []
    date_lst = []

    for pg in range(1,page+1):

        page_num = pg *10 - 9

        url = f'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={corp}&sort=0&photo=0&field=0&pd=3&ds={bgn_date1}&de={end_date1}&cluster_rank=24&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{bgn_date2}to{end_date2},a:all&start={page_num}'
        res = requests.get(url)
        soup = BeautifulSoup(res.text , 'html.parser')
        lis = soup.select('#main_pack > section > div > div.group_news > ul>li')

        for li in lis:
            #제목
            title = li.select('div.news_wrap.api_ani_send > div > a')[0].text

            title_lst.append(title)

            # url
            url_path = li.select('div.news_wrap.api_ani_send > div > a')[0]['href']
            url_lst.append(url_path)

            #날짜

            if len(li.select('div.news_info > div.info_group > span'))==1:
                date = li.select('div.news_info > div.info_group > span')[0].text
                date_lst.append(date)


            if len(li.select('div.news_info > div.info_group > span'))==2:
                date = li.select('div.news_info > div.info_group > span')[1].text
                date_lst.append(date)
    
    df = pd.DataFrame({'날짜':date_lst,'뉴스제목':title_lst,'url':url_lst})
    
    output_result = ''
    for i in range(len(df)):
        title = df['뉴스제목'].iloc[i]
        news_url = df['url'].iloc[i]
        output_result += title + "\n" + news_url + "\n\n"
        if i == num:
            break
        
    return output_result

def finance_gpt(user_text):
    
    with torch.no_grad():
        answer = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + user_text + SENT + A_TKN + answer)).unsqueeze(dim=0)
            pred = model1(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            answer += gen.replace("▁", " ").strip()
    return answer
####################################
 
######## 텔레그램 관련 코드 ########
token = "5403110188:AAEbcgi6cDNmgdRHERhGhprFQgMUHzi-rtI"
id = 5322933876
 
bot = telegram.Bot(token)
info_message = '''안녕하세요 금융챗봇 고슴도치입니다'''
bot.sendMessage(chat_id=id, text=info_message)
 
updater = Updater(token=token, use_context=True)
dispatcher = updater.dispatcher
updater.start_polling()
 

In [3]:
### 챗봇 답장

def handler(update, context): 
    user_text = update.message.text # 사용자가 보낸 메세지 user_text 변수에 저장
    user_words = user_text.split()
    
    # 종목명 찾기
    for word in user_words:
        # 종목명을 포함한다면
        if word in corp_list.tolist():
            corp = [w for w in user_words if w in corp_list.tolist()][0]
            break
        else:
            corp='종목명 없음'

    # 종목명 없으면 gpt or 추천
    if corp == '종목명 없음':
        # 살만한 주식 뭐있어?
        if user_text in reco_lst1:
            bot.send_message(chat_id=id, text='고슴도치가 추천해드리겠습니다') # 답장 보내기
            
        # gpt일반대화
        else:
            answer = finance_gpt(user_text)
            bot.send_message(chat_id=id, text=answer) # 답장 보내기
    
    # 종목명 있으면
    else:
        for word in user_words:
            # 최신뉴스 최근뉴스 포함시
            if word in content_lst:
                crawl_news = crawl_news(corp)
                bot.send_message(chat_id=id, text=crawl_news)
                break

echo_handler = MessageHandler(Filters.text, handler)
dispatcher.add_handler(echo_handler)
####################################

In [38]:
user_text = '최신뉴스 알려줘'
user_words = user_text.split()
user_words

['최신뉴스', '알려줘']

In [39]:
corp

'종목명 없음'

In [42]:
# 종목명 찾기
for word in user_words:
    # 종목명을 포함한다면
    if word in corp_list.tolist():
        corp = [w for w in user_words if w in corp_list.tolist()][0]
        break
    else:
        corp='종목명 없음'
        break
corp

'종목명 없음'

In [43]:
# 종목명 없으면 gpt or 추천
if corp == '종목명 없음':
    # 살만한 주식 뭐있어?
    if user_text in reco_lst1:
        bot.send_message(chat_id=id, text='고슴도치가 추천해드리겠습니다') # 답장 보내기

    # gpt일반대화
    else:
        answer = finance_gpt(user_text)
        bot.send_message(chat_id=id, text=answer) # 답장 보내기

# 종목명 있으면
else:
    for word in user_words:
        # 최신뉴스 최근뉴스 포함시
        if word in content_lst:
            crawl_news = crawl_news(corp)
            bot.send_message(chat_id=id, text=crawl_news)
            break

Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running


KeyboardInterrupt: 

In [7]:
finance_gpt(user_text)

KeyboardInterrupt: 

In [5]:
import math
import numpy as np
import pandas as pd
import re
import os

# telegram
import telegram as tel
from telegram.ext import Updater
from telegram.ext import MessageHandler, Filters
from telegram import chat

# 크롤링
from bs4 import BeautifulSoup

import requests
from selenium import webdriver

# gpt함수
import torch
import urllib.request
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel
 
######## 크롤링 관련 함수 ########

def crawl_news(corp,page=1,num=5,bgn_date='2022.03.01',end_date='2022.03.30'):
    
    bgn_date1 = bgn_date
    bgn_date2 = bgn_date.replace('.','')
    end_date1 = end_date
    end_date2 = end_date.replace('.','')
    
    title_lst = []
    url_lst = []
    date_lst = []

    for pg in range(1,page+1):

        page_num = pg *10 - 9

        url = f'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={corp}&sort=0&photo=0&field=0&pd=3&ds={bgn_date1}&de={end_date1}&cluster_rank=24&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{bgn_date2}to{end_date2},a:all&start={page_num}'
        res = requests.get(url)
        soup = BeautifulSoup(res.text , 'html.parser')
        lis = soup.select('#main_pack > section > div > div.group_news > ul>li')

        for li in lis:
            #제목
            title = li.select('div.news_wrap.api_ani_send > div > a')[0].text

            title_lst.append(title)

            # url
            url_path = li.select('div.news_wrap.api_ani_send > div > a')[0]['href']
            url_lst.append(url_path)

            #날짜

            if len(li.select('div.news_info > div.info_group > span'))==1:
                date = li.select('div.news_info > div.info_group > span')[0].text
                date_lst.append(date)


            if len(li.select('div.news_info > div.info_group > span'))==2:
                date = li.select('div.news_info > div.info_group > span')[1].text
                date_lst.append(date)
    
    df = pd.DataFrame({'날짜':date_lst,'뉴스제목':title_lst,'url':url_lst})
    
    output_result = ''
    for i in range(len(df)):
        title = df['뉴스제목'].iloc[i]
        news_url = df['url'].iloc[i]
        output_result += title + "\n" + news_url + "\n\n"
        if i == num:
            break
        
    return output_result

def finance_gpt(user_text):
    user_text = update.message.text
    
    with torch.no_grad():
        answer = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + user_text + SENT + A_TKN + answer)).unsqueeze(dim=0)
            pred = model1(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            answer += gen.replace("▁", " ").strip()
    return answer
####################################
 
######## 텔레그램 관련 코드 ########
token = "5403110188:AAEbcgi6cDNmgdRHERhGhprFQgMUHzi-rtI"
id = 5322933876
 
bot = telegram.Bot(token)
info_message = '''안녕하세요 금융챗봇 고슴도치입니다'''
bot.sendMessage(chat_id=id, text=info_message)
 
updater = Updater(token=token, use_context=True)
dispatcher = updater.dispatcher
updater.start_polling()
 
### 챗봇 답장

def handler(update, context): 
    user_text = update.message.text # 사용자가 보낸 메세지 user_text 변수에 저장
    user_words = user_text.split()
    
    for word in user_words:
        # 종목명을 포함한다면
        if word in corp_list.tolist():
            corp = [w for w in user_words if w in corp_list.tolist()]
            user_words1 = user_words.remove(corp)
            
            # 최신뉴스, 최근뉴스 포함한다면
            for word1 in user_words1:
                if word1 in content_lst:
                    crawl_news = crawl_news(corp)
                    bot.send_message(chat_id=id, text=crawl_news)
                    break
          
        
        # 종목명을 포함하지 않는경우
        else:
            #1: 주식 추천해줘
            if user_text in reco_lst1:
                bot.send_message(chat_id=id, text='고슴도치가 추천해드리겠습니다') # 답장 보내기
                break
#             elif

            else:    
                answer = finance_gpt(user_text)
                bot.send_message(chat_id=id, text=answer) # 답장 보내기
                break
            
 
echo_handler = MessageHandler(Filters.text, handler)
dispatcher.add_handler(echo_handler)
####################################